In [ ]:
from selenium import webdriver

#check google chrome version:chrome://settings/help
#source: https://pypi.org/project/chromedriver-binary/92.0.4515.107.0/
import chromedriver_binary
from time import sleep
import requests

In [ ]:
#source: https://www.worthwebscraping.com/how-to-scrape-linkedin-profiles-without-login-using-python/
#does not work so far - 2021-08-31
#number of seconds to sleep
timeSleep = 3

driver = webdriver.Chrome()
sleep(timeSleep)
driver.maximize_window()
sleep(timeSleep)
driver.get("https://www.linkedin.com/")
sleep(timeSleep)

#save cookies
cookies_dict = {}
for cookie in driver.get_cookies():
    cookies_dict[cookie['name']] = cookie['value']

driver.close()

sleep(timeSleep)

response = requests.get("https://www.linkedin.com/company/twitter",
                       cookies = cookies_dict,
                        headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
                              'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
                                                   'accept-encoding': 'gzip, deflate, br',
                                                   'accept-language': 'en-US,en;q=0.9',
                                                   'upgrade-insecure-requests': '1',
                                                   'scheme': 'https'})

html = response.text

HtmlPath = "linkedIn1.html"
page_fun = open(HtmlPath,'w',encoding='utf-8')
page_fun.write(html)
page_fun.close()

### Use package = scrape_linkedin

In [1]:
import pandas as pd
import requests
from time import sleep
from selenium import webdriver
import chromedriver_binary
from scrape_linkedin import ProfileScraper
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [5]:
#created a linkedin profile: yeomangoh@yahoo.com, password:08075323
#get li_at cookie (need login): https://www.smartwriter.ai/university/how-to-access-your-linkedin-cookie-and-what-exactly-does-a-cookie-do
li_at_value = 'AQEDATd4RK4C9QUEAAABe5xdggMAAAF7wGoGA1YAP8VFRmt1hVKulFeHCP-2ztYgJis24Yuq_Bxd7-tMlg8g3NG06cHybRecFh3tJjtGKT2l8s3fWVbctPC2z22jnR0oUFjhuL0-c0fF8FPzku2jlYTx'

#git location
# C:\Users\JiunShyanGoh\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Git

#define chromedriver
# driver = webdriver.Chrome()

#package location: scrape-linkedin-selenium
#https://github.com/austinoboyle/scrape-linkedin-selenium

# with ProfileScraper(cookie = li_at_value) as scraper:
#     profile = scraper.scrape(user='austinoboyle')
# print(profile.to_dict())

In [ ]:
#example
river = webdriver.Chrome()
with ProfileScraper(cookie = li_at_value) as scraper:
    profile = scraper.scrape(user='austinoboyle')

profileDict = profile.to_dict()
resultList = []

#personal info
colList = ['name', 'phone', 'email', 'company', 'school']
resultDict = {}
for col in colList:
    if col in profileDict['personal_info'].keys():
        resultDict[col] = profileDict['personal_info'][col]
        
#working exp
#current job
resultDict['current title'] = profileDict['experiences']['jobs'][0]['title']

#skill
resultDict['skills'] = ','.join([i['name'] for i in profileDict['skills']])

pd.DataFrame([resultDict])

In [10]:
#get names from organization page
url = "https://healthaxis.com/leadership-team"

options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome(options = options)
driver.get(url)

className = "wpb_text_column wpb_content_element "
a_Tag = driver.find_elements_by_xpath("//div[@class = '%s']/div/p/span/a"%className)
hrefList = [i.get_attribute("href") for i in a_Tag]

#clean href 
nameList = [re.sub("-", " ", re.search(".*/(\w*-\w*).*/", i).group(1)) for i in hrefList]
userNameList = [re.sub(" ", "", i) for i in nameList]

In [ ]:
resultList = []
failedList = []

for userName in userNameList:
    
    try:
        with ProfileScraper(cookie = li_at_value) as scraper:
            profile = scraper.scrape(user ='%s'%userName)

        profileDict = profile.to_dict()

        #personal info
        colList = ['name', 'phone', 'email', 'company', 'school']
        resultDict = {}
        for col in colList:
            if col in profileDict['personal_info'].keys():
                resultDict[col] = profileDict['personal_info'][col]

        #working exp
        #current job
        resultDict['current title'] = profileDict['experiences']['jobs'][0]['title']

        #skill
        resultDict['skills'] = ','.join([i['name'] for i in profileDict['skills']])

        #store result
        resultList.append(resultDict)
    
    except:
        failedList.append(name)

In [ ]:
dfResult = pd.DataFrame(resultList)
dfResult

In [20]:
timeSleep = 3

#search and click
url = 'https://www.linkedin.com'
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome()
driver.get(url)
# driver.add_cookie({"name":"li_at", "value":li_at_value})

email = "yeomangoh@yahoo.com"
password = "08075323"
#key in email and password
emailBox = driver.find_element_by_xpath("//*[@id = 'session_key']")
emailBox.send_keys(email)
sleep(timeSleep)
passwordBox = driver.find_element_by_xpath("//*[@id = 'session_password']")
passwordBox.send_keys(password)

#click sign in button
sleep(timeSleep)
signInButton = driver.find_element_by_xpath("//button[@type = 'submit']")
signInButton.click()

#click skip button when ask phone number
# sleep(timeSleep)
# skipButton = driver.find_element_by_xpath("//button[@class = 'secondary-action']")
# skipButton.click()

#key in text
sleep(timeSleep)
companyName = "HealthAxis"
#example where has result
name = "jim Clark"
#example where no result
# name = "michele mahoney"
searchText = name + ' ' + companyName
inputBox_xPath = "//input[@placeholder = 'Search']"
wait = WebDriverWait(driver, 10)
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, inputBox_xPath)))
inputBox = driver.find_element_by_xpath(inputBox_xPath)
inputBox.send_keys("%s"%searchText)
inputBox.send_keys(Keys.RETURN)

#click result link
sleep(timeSleep)
resultLink_xPath = "//a[@class = 'app-aware-link']"
#here may fail if no results even fuzzy
wait = WebDriverWait(driver, 10)
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, resultLink_xPath)))
resultLink = driver.find_element_by_xpath(resultLink_xPath)
# resultLink.click()

#get url from href to pass to scrape_linkedin func:
currentUrl = resultLink.get_attribute("href")
userName = re.search(".*/(.*)\?", currentUrl).group(1)

#example of href where no exact search result
#href:https://www.linkedin.com/search/results/all/?keywords=michele%20mahoney%20healthaxis&origin=GLOBAL_SEARCH_HEADER&sid=-5R
if re.search("results\/all", currentUrl):
    print("No result")
else:
    print("Result found")

driver.close()

Result found


In [7]:
with ProfileScraper(cookie = li_at_value) as scraper:
    profile = scraper.scrape(user ='%s'%userName)

    profileDict = profile.to_dict()

    #personal info
    colList = ['name', 'phone', 'email', 'company', 'school']
    resultDict = {}
    for col in colList:
        if col in profileDict['personal_info'].keys():
            resultDict[col] = profileDict['personal_info'][col]

    #working exp
    #current job
    resultDict['current title'] = profileDict['experiences']['jobs'][0]['title']

    #skill
    resultDict['skills'] = ','.join([i['name'] for i in profileDict['skills']])
resultDict

{'name': 'Jim Clark',
 'phone': None,
 'email': None,
 'company': 'HealthAxis Group',
 'school': 'Florida State University - College of Business',
 'current title': 'Chief Growth Officer',
 'skills': 'Start-ups,Management,Business Development,New Business Development,Strategic Planning,Sales,Mergers & Acquisitions,Leadership,Team Building,Marketing,Salesforce.com,Executive Management,Strategic Partnerships,Healthcare,Sales Management,Entrepreneurship,Marketing Strategy,SaaS,Contract Negotiation,Business Strategy,Product Development,Mobile Devices,Solution Selling,Business Planning,Product Management,CRM,Software as a Service (SaaS),Business Process Improvement,Strategy,Due Diligence'}

### Debug

In [ ]:
with ProfileScraper(cookie = li_at_value, timeout = 10) as scraper:
    profile = scraper.scrape(user ='%s'%nameList[1])

profileDict = profile.to_dict()

#personal info
colList = ['name', 'phone', 'email', 'company', 'school']
resultDict = {}
for col in colList:
    if col in profileDict['personal_info'].keys():
        resultDict[col] = profileDict['personal_info'][col]

#working exp
#current job
resultDict['current title'] = profileDict['experiences']['jobs'][0]['title']

#skill
resultDict['skills'] = ','.join([i['name'] for i in profileDict['skills']])

#store result
resultList.append(resultDict)